In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 

In [3]:
import pandas as pd
import torch
import numpy as np

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available! Running on GPU.')
else:
    device = torch.device('cpu')
    print('CUDA is not available. Running on CPU.')

# Set a random seed for reproducibility
SEED = 2137
torch.manual_seed(SEED)
CLASSES = 5
# Clear any cached memory
torch.cuda.empty_cache()

# Check if CUDA is available
print('Is CUDA available?', torch.cuda.is_available())

CUDA is available! Running on GPU.
Is CUDA available? True


In [4]:
!nvidia-smi

Sat Apr 20 16:14:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [6]:
data = pd.read_csv("/content/drive/MyDrive/data_processed_balanced_125k.txt", header = 0)
data.head()


,text,rating
0,We got tricked by the name Wyndham attached to...,0
1,"Very nice view indeed for this hotel, and pret...",0
2,We blocked out 10 rooms for a wedding and I ca...,0
3,I normally don't bother with reviews but this ...,0
4,stayed in this hotel last year at easter it wa...,0


In [7]:
data['rating'].value_counts()

rating
0    25000
1    25000
2    25000
3    25000
4    25000
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
# Create empty DataFrames for train and validation sets
train_data = pd.DataFrame(columns=['text', 'rating'])
valid_data = pd.DataFrame(columns=['text', 'rating'])

# Split the data evenly
for class_label in range(CLASSES):
    # Extract examples of current class
    class_df = data[data['rating'] == class_label]

    # Split examples of current class into train and validation sets
    class_train, class_valid = train_test_split(class_df, test_size=0.08, random_state=42)

    # Append train examples to train_df
    train_data = pd.concat([train_data, class_train])

    # Append validation examples to valid_df
    valid_data = pd.concat([valid_data, class_valid])

  # Reset index for both DataFrames
train_data.reset_index(drop=True, inplace=True)
valid_data.reset_index(drop=True, inplace=True)

# Ensure balance across classes in train and validation sets
train_class_counts = train_data['rating'].value_counts()
valid_class_counts = valid_data['rating'].value_counts()

print("Train class counts: \n")
print(train_class_counts)

print("Validation class counts:")
print(valid_class_counts)

Train class counts: 

rating
0    23000
1    23000
2    23000
3    23000
4    23000
Name: count, dtype: int64
Validation class counts:
rating
0    2000
1    2000
2    2000
3    2000
4    2000
Name: count, dtype: int64


In [9]:
# shaffle the data

train_data = train_data.sample(frac=1, random_state=42).reset_index(drop=True)

valid_data = valid_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [10]:
print(train_data.head(10))
train_data['rating'].value_counts()

                                                text rating
0  We stayed in the family room (1 double, 1 sing...      3
1  My boyfriend and I stayed here our first night...      4
2  We stayed as 4 people in 4 rooms playing golf ...      4
3  The small dark beige coloured rooms with lumpy...      1
4  Let me start by saying I have been to Cancun s...      1
5  Stayed here for a night, had dinner and breakf...      2
6  After our wedding on the 4th of June 2016, we ...      4
7  This hotel has poor hygiene, bathrooms unclean...      0
8  Lovely hotel. Rooms were very spacious and wel...      3
9  We stayed here for 4 nights. Cons: the wifi wa...      2


rating
3    23000
4    23000
1    23000
2    23000
0    23000
Name: count, dtype: int64

In [11]:
print(valid_data.head(10))
valid_data['rating'].value_counts()

                                                text rating
0  Travelled in March 2017 arriving on the Saturd...      3
1  Stayed one night as in transit. The location i...      2
2  This hotel is trying to charge my card 4 times...      0
3  Very good location to the casinos. Rooms are a...      2
4  My boyfriend and I stayed at this resort in th...      2
5  We really enjoyed our short break at East Thor...      3
6  These people cover the butts by trying to make...      0
7  Purpose: I stayed in the hotel for 4 nights an...      2
8  Stayed here for only 1 night after landed in J...      3
9  I am totally confused how anyone could write a...      0


rating
3    2000
2    2000
0    2000
1    2000
4    2000
Name: count, dtype: int64

In [12]:
from simpletransformers.classification import ClassificationModel
CLASSES = 5
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=CLASSES, use_cuda = True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [13]:
model.train_model(train_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/230 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/14375 [00:00<?, ?it/s]

(14375, 0.9276844291355298)

In [14]:
predictions, raw_outputs = model.predict(["I would love to burn this place to the ground! The food was uneatable, please close this place.", "Best place to ever be in, 10 outta ten my man!"])
print(predictions, raw_outputs)

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[0 4] [[ 4.77734375  1.49511719 -1.36035156 -3.46289062 -2.89257812]
 [-0.85888672 -2.4609375  -2.06835938  0.92431641  4.35546875]]


In [15]:
# Make predictions on the validation data
predictions, raw_outputs = model.predict(valid_data['text'].tolist())


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
# Evaluate the predictions
# Assuming 'labels' are the actual labels in your validation data
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print()

print(confusion_matrix(valid_data['rating'].tolist(), predictions))
print(classification_report(valid_data['rating'].tolist(), predictions))


[[1491  450   50    3    6]
 [ 550 1059  348   30   13]
 [ 105  422 1093  349   31]
 [  10   25  322 1193  450]
 [   7    3   14  426 1550]]
              precision    recall  f1-score   support

           0       0.69      0.75      0.72      2000
           1       0.54      0.53      0.53      2000
           2       0.60      0.55      0.57      2000
           3       0.60      0.60      0.60      2000
           4       0.76      0.78      0.77      2000

    accuracy                           0.64     10000
   macro avg       0.64      0.64      0.64     10000
weighted avg       0.64      0.64      0.64     10000



In [17]:
# SAVE MODEL

output_dir = "outputs/checkpoint-22500-epoch-1"
model.save_model(output_dir)

import os
import shutil
save_dir = "model_18_35_12_03_2024"

In [18]:
## EXPORT
save_dir = "model_21_57_13_03_2024_69ACC"

shutil.copytree(output_dir, "/content/drive/My Drive/" + save_dir)

'/content/drive/My Drive/model_21_57_13_03_2024_69ACC'

In [19]:
## IMPORT

# Copy the model directory from Google Drive to Colab

imported_dir = "imported_model_the_one"

shutil.copytree("/content/drive/My Drive/" + save_dir, imported_dir)

from simpletransformers.classification import ClassificationModel

# Load the model
model = ClassificationModel('roberta', imported_dir, use_cuda=True)

OSError: imported_model_the_one does not appear to have a file named config.json. Checkout 'https://huggingface.co/imported_model_the_one/main' for available files.